In [1]:
import requests
import json
from bs4 import BeautifulSoup
import re
import pandas as pd

In [2]:
def find_interval(down, df, column, row):
    current_row = row
    while current_row < df.shape[0] - 1 and current_row > 0 and df.iloc[current_row][column] == "":
        if down:
            current_row = current_row + 1
        else:
            current_row = current_row - 1
    return current_row

In [3]:
def clean_data(df):
    for country in range(df.shape[1]):
        for row in range(df.shape[0]):
            if df.iloc[row][country] == "":
                if row == 0:
                    down_interval = find_interval(True, df, country, row+1)
                    df.iloc[row][country] = df.iloc[down_interval][country]
                if row == df.shape[0] - 1:
                    up_interval = find_interval(False, df, country, row-1)
                    df.iloc[row][country] = df.iloc[up_interval][country]
                else:
                    down_interval = find_interval(True, df, country, row)
                    up_interval = find_interval(False, df, country, row)
                    up = df.iloc[up_interval][country]
                    down = df.iloc[down_interval][country]
                    
                    if df.iloc[up_interval][country] == "":
                        up = df.iloc[down_interval][country]
                    if df.iloc[down_interval][country] == "":
                        down = df.iloc[up_interval][country]

                    average = (float(up) + float(down)) / 2
                    df.iloc[row][country] = average
    return df

In [28]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import time

# configure webdriver
options = Options()
options.headless = True  
options.add_argument("--window-size=1920,1080")  # set window size to native GUI size
options.add_argument("start-maximized")  # make the window full-screen

driver = webdriver.Chrome(options=options)
driver.get("https://data.oecd.org/gga/trust-in-government.htm")

# click table button
table_button = driver.find_element(By.XPATH, "//a[@class='chart-types-button' and @data-type='table']")
table_button.click()
checkbox = driver.find_element(By.CLASS_NAME, "use-latest-data-checkbox")
if checkbox.is_selected():
    # If checkbox is selected, click it to uncheck it
    checkbox.click()
slider = driver.find_element(By.CLASS_NAME, 'noUi-handle-lower')
slider_width = slider.size['width']
slider_position = slider.location['x']

container_width = 1000
actions = ActionChains(driver)
# move the slider all the way to the left (0%)
actions.click_and_hold(slider).move_by_offset(-int(container_width * 0.8125), 0).release().perform()

time.sleep(5)

In [83]:
gov_trust_dict = {}
page_source = driver.page_source
soup = BeautifulSoup(page_source)
years_html = soup.find(class_="table-chart-thead")
# extract all numerical values
years = re.findall(r'\d+\.?\d*', years_html.text)
table_html = soup.find_all(class_="table-chart-tbody-tr")

for row in table_html:
    data_values = []
    location = row.find(class_="table-chart-tbody-th")
    data_html = row.find_all(class_="table-chart-td")
    for data in data_html:
        data_values.append(data.text)
    gov_trust_dict[location.text] = data_values
gov_trust_df = pd.DataFrame(gov_trust_dict, index=years)

In [84]:
gov_trust_df = clean_data(gov_trust_df)
gov_trust_df = gov_trust_df.T
gov_trust_df.reset_index(inplace=True)
gov_trust_df.rename(columns={'index': 'country'}, inplace=True)
gov_trust_df

,country,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Australia,53.2,52.7,64.5,62.75,61.0,53.1,42.0,45.6,46.5,47.9,45.3,45.3,46.9,46.9,44.6,51.9,49.9
1,Austria,49.7,37.7,25.7,48.1,49.2,40.7,37.7,41.7,40.8,45.5,43.3,43.6,48.9,51.2,62.6,61.0,61.0
2,Belgium,55.2,60.2,46.3,40.0,33.7,28.9,44.0,55.4,46.9,45.9,41.9,45.0,44.2,32.8,29.5,47.3,57.2
3,Brazil,34.1,38.3,51.1,42.7,51.1,50.3,45.6,33.4,35.5,19.9,26.4,16.5,16.8,34.1,36.2,32.7,39.2
4,Canada,44.3,63.8,59.0,60.8,55.1,55.3,52.3,50.6,51.7,64.4,61.8,65.3,61.0,54.9,60.0,61.0,50.7
5,Chile,58.1,42.6,40.5,57.5,47.9,32.3,34.3,30.7,40.0,30.0,20.4,26.8,33.5,15.3,17.1,23.2,28.7
6,Colombia,48.1,51.2,63.5,44.0,55.2,44.6,35.7,29.3,29.8,27.2,26.1,22.2,27.5,32.8,37.4,28.9,29.7
7,Costa Rica,37.9,44.6,36.1,52.1,53.3,32.3,28.0,26.2,40.4,26.1,27.9,40.7,48.0,28.3,29.8,31.5,60.0
8,Czechia,28.3,26.9,31.2,35.5,31.2,20.6,17.0,24.3,34.0,43.5,41.5,34.4,42.1,37.0,31.9,28.4,34.1
9,Denmark,66.9,59.3,66.6,62.7,58.7,47.2,53.4,39.1,45.8,58.0,46.8,57.2,63.2,63.3,71.6,65.2,63.5


In [85]:
gov_trust_df = gov_trust_df.melt(id_vars=['country'], var_name='year', value_name='public_trust_percentage')
gov_trust_df
gov_trust_df.to_csv('gov_trust.csv', index=False)